## Quant/Data Science Project Predicting the Stock Market

Current ideas: Obviously machine learning binary reggresion could be good or random forest. Supervised learning since I know the outcome of price tomorrow, or just up or down.
Research bayes theorem for predicitng price and game theory because it appartently is useful.

In [100]:
import yfinance as yf
import quandl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import math
import zipline as zp
import pyfolio as pf
import pandas as pd

In [28]:
quandl.ApiConfig.api_key = 'Cnf7GU-eiNQJspGFL3Dx'

In [126]:
data = yf.download("AAPL", start="2017-12-31", end="2024-12-31")
print(data.head())

[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2018-01-02  40.524338  40.533750  39.818585  40.030313  102223600
2018-01-03  40.517284  41.063069  40.453769  40.587860  118071600
2018-01-04  40.705486  40.808997  40.481999  40.590212   89738400
2018-01-05  41.168938  41.255979  40.710199  40.801947   94640000
2018-01-08  41.016022  41.312437  40.917213  41.016022   82271200


In [136]:
table = pd.DataFrame(data)

In [138]:
table

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2018-01-02,40.524338,40.533750,39.818585,40.030313,102223600
2018-01-03,40.517284,41.063069,40.453769,40.587860,118071600
2018-01-04,40.705486,40.808997,40.481999,40.590212,89738400
2018-01-05,41.168938,41.255979,40.710199,40.801947,94640000
2018-01-08,41.016022,41.312437,40.917213,41.016022,82271200
...,...,...,...,...,...
2024-12-23,255.270004,255.649994,253.449997,254.770004,40858800
2024-12-24,258.200012,258.209991,255.289993,255.490005,23234700


In [212]:
table['Result'] = np.where(table['Close'] - table['Open'] <= 0, 0, 1)

In [216]:
table.head(50)

Price,Close,High,Low,Open,Volume,Result
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,
Date,,,,,,
2018-01-02,40.524338,40.533750,39.818585,40.030313,102223600,1
2018-01-03,40.517284,41.063069,40.453769,40.587860,118071600,0
2018-01-04,40.705486,40.808997,40.481999,40.590212,89738400,1
2018-01-05,41.168938,41.255979,40.710199,40.801947,94640000,1
2018-01-08,41.016022,41.312437,40.917213,41.016022,82271200,0
2018-01-09,41.011326,41.183058,40.794895,41.063081,86336000,0
2018-01-10,41.001900,41.004255,40.698427,40.736069,95839600,1
2018-01-11,41.234810,41.284214,41.048963,41.072486,74670800,1


In [218]:
Result = table['Result']

In [222]:
Result

Date
2018-01-02    1
2018-01-03    0
2018-01-04    1
2018-01-05    1
2018-01-08    0
             ..
2024-12-23    1
2024-12-24    1
2024-12-26    1
2024-12-27    0
2024-12-30    0
Name: Result, Length: 1760, dtype: int32

In [224]:
Features = table.drop('Result', axis=1)

In [226]:
Features

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2018-01-02,40.524338,40.533750,39.818585,40.030313,102223600
2018-01-03,40.517284,41.063069,40.453769,40.587860,118071600
2018-01-04,40.705486,40.808997,40.481999,40.590212,89738400
2018-01-05,41.168938,41.255979,40.710199,40.801947,94640000
2018-01-08,41.016022,41.312437,40.917213,41.016022,82271200
...,...,...,...,...,...
2024-12-23,255.270004,255.649994,253.449997,254.770004,40858800
2024-12-24,258.200012,258.209991,255.289993,255.490005,23234700


In [228]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, log_loss, \
roc_curve, auc, precision_recall_curve

In [230]:
logreg = LogisticRegression()
scaler = StandardScaler()
pipeline = Pipeline([
    ('scaler', scaler),
    ('logreg', logreg)
])

In [232]:
X_train, X_test, y_train, y_test = train_test_split(Features, Result, test_size = 0.2, random_state = 4)

In [234]:
logreg.fit(X_train, y_train)

LogisticRegression()

In [236]:
logreg.predict(X_test)

array([1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,

In [238]:
print(logreg.score(X_train, y_train))

0.5340909090909091


In [240]:
print(logreg.score(X_test, y_test))

0.5113636363636364
